In [1]:
from DB.chromadb_storing import ChromaDB
from langchain.schema import Document
from utils import get_only_paragraphs, get_all_datas, create_documents
BASE_DIR = '../../datas' # 데이터가 저장돼 있는 루트를 의미합니다.

df = get_only_paragraphs(BASE_DIR)
# df = get_all_datas(BASE_DIR) # 이미지와 테이블이 포함된 전체 데이터도 가져올 수 있습니다.
df[:3]


,id,type,image_route,dir_route,file_name,page,investment,company_name,table,summary,original_content
3,12,paragraph,NaN,datas/CJ제일배당_교보증권(2024.11.13),CJ제일배당_교보증권(2024.11.13).pdf,1,교보증권(2024.11.13),CJ제일배당,NaN,"CJ제일제당의 3분기 연결 매출액과 영업이익은 각각 4조 6,204억 원(YoY -...","대한통운을 제외한 3분기 연결 매출 및 영업이익은 각각 4조 6,204억원(YoY ..."
4,13,paragraph,NaN,datas/CJ제일배당_교보증권(2024.11.13),CJ제일배당_교보증권(2024.11.13).pdf,1,교보증권(2024.11.13),CJ제일배당,NaN,CJ제일제당의 국내 식품 매출은 소비 둔화와 경쟁 심화로 인해 YoY -6%를 기록...,"① 식품 매출 및 이익은 각각 4조 6,204억원(YoY -1.1%), 1,613억..."
6,16,paragraph,NaN,datas/CJ제일배당_교보증권(2024.11.13),CJ제일배당_교보증권(2024.11.13).pdf,1,교보증권(2024.11.13),CJ제일배당,NaN,"CJ제일제당의 목표주가를 41만원으로 하향 조정하며, 3분기 성수기 시즌임에도 불구...",CJ제일제당의 목표주가를 41만원으로 하향 조정. 3분기 성수기 시즌에도 불구하고 ...


In [2]:
# 데이터프레임의 행들을 각각 Document타입으로 바꾼 뒤, 리스트에 담습니다.
documents = create_documents(df)
documents[:3]

[Document(metadata={'id': 12, 'type': 'paragraph', 'image_route': nan, 'dir_route': 'datas/CJ제일배당_교보증권(2024.11.13)', 'file_name': 'CJ제일배당_교보증권(2024.11.13).pdf', 'page': 1, 'investment': '교보증권(2024.11.13)', 'company_name': 'CJ제일배당', 'table': nan, 'original_content': '대한통운을 제외한 3분기 연결 매출 및 영업이익은 각각 4조 6,204억원(YoY -1%), 2,764 억원(YoY 0.4%) 시현. 연결 이익은 시장 기대치 부합했으나, 대통 제외시 시장 기대치 - 7% 하회. 식품 영업이익이 예상대비 부진했고(시장 컨센 YoY -10%대 Vs. 실제 -30% 대), 이를 바이오/F&C가 상쇄.'}, page_content='CJ제일제당의 3분기 연결 매출액과 영업이익은 각각 4조 6,204억 원(YoY -1%), 2,764억 원(YoY 0.4%)으로 발표됐다. 이는 전년 대비 소폭 감소한 수치이며, 식품 부문에서 영업이익이 예상보다 크게 하락했다.'),
 Document(metadata={'id': 13, 'type': 'paragraph', 'image_route': nan, 'dir_route': 'datas/CJ제일배당_교보증권(2024.11.13)', 'file_name': 'CJ제일배당_교보증권(2024.11.13).pdf', 'page': 1, 'investment': '교보증권(2024.11.13)', 'company_name': 'CJ제일배당', 'table': nan, 'original_content': '① 식품 매출 및 이익은 각각 4조 6,204억원(YoY -1.1%), 1,613억원(YoY -31%)을 시

In [3]:
# add_docs를 통해 문서들을 저장합니다.

collection_name = 'chrdb.db'
DB = ChromaDB(collection_name)
DB.add_docs(documents)



In [4]:
# verify_db를 통해 db내의 데이터 수를 확인할 수 있고, db객체를 받을 수 있습니다.
db = DB.verify_db()


Total documents in collection 'chrdb.db': 388


In [5]:
# 데이터 저장
# DB.save_collection()

In [6]:
# 데이터 로드
# DB.load_collection()

In [7]:
# 위처럼 import하여 kiwi tokenizer을 사용하는 bm25와 dpr, 앙상블 리트리버를 생성할 수 있습니다.
from retrievals import bm25, dpr, ensemble

topk = 3

DPRRetriever = dpr(db, topk = 3)
BM25Retriever = bm25(documents, topk = 3)

retrievals = [DPRRetriever, BM25Retriever]
weights = [0.5, 0.5]
search_type = 'mmr'
ensemble_retriever = ensemble(retrievals, topk = topk, weights = weights, search_type = search_type)



In [8]:
# 다음과 같이 retriever.invoke(query)를 통해 관련 문서를 검색할 수 있습니다.
query = '네이버의 2024년 매출을 알려줘.'

DPRRetriever.invoke(query)

[Document(metadata={'company_name': '네이버', 'dir_route': 'datas/네이버_하나증권(2024.10.10)', 'file_name': '네이버_하나증권(2024.10.10).pdf', 'id': 19, 'investment': '하나증권(2024.10.10)', 'original_content': '네이버는 2024년 연결 기준 영업수익 10조 5,184억원(+8.8%YoY), 영업이익 1조 8,994억원 (+27.6%YoY, OPM 18.1%)을 기록할 것으로 전망한다. 서치플랫폼 매출액은 3조 8,546억원 (+7.4%YoY)으로 추정한다. DA 광고 지면 증가 효과는 4분기까지 높은 성장률로 확인될 전 망이다. 커머스 매출액은 2조 9,045억원(+14.1%YoY)으로 전망한다. 네이버는 연내 모바일 쇼핑 탭을 네이버플러스 스토어로 개편할 계획이다. 스마트/브랜드스토어를 중심으로한 개 인화 추천, UI/UX 개선이 도입될 예정이다. 성장 둔화를 겪고 있는 커머스 광고 매출과 커 머스 GMV의 반등을 기대한다.', 'page': 1, 'type': 'paragraph'}, page_content='네이버는 2024년 연결 기준 영업수익 10조 5,184억원, 영업이익 1조 8,994억원을 기록할 것으로 전망하며, 특히 쇼핑 탭을 네이버플러스 스토어로 개편하여 개인화 추천 및 UI/UX 개선을 통해 커머스 광고 매출과 GMV의 반등을 기대함.'),
 Document(metadata={'company_name': '네이버', 'dir_route': 'datas/네이버_하나증권(2024.10.10)', 'file_name': '네이버_하나증권(2024.10.10).pdf', 'id': 45, 'investment': '하나증권(2024.10.10)', 'original_content': '주: 2Q21 왓패드 인수, 4Q21

In [9]:
BM25Retriever.invoke(query)

[Document(metadata={'id': 19, 'type': 'paragraph', 'image_route': nan, 'dir_route': 'datas/네이버_하나증권(2024.10.10)', 'file_name': '네이버_하나증권(2024.10.10).pdf', 'page': 1, 'investment': '하나증권(2024.10.10)', 'company_name': '네이버', 'table': nan, 'original_content': '네이버는 2024년 연결 기준 영업수익 10조 5,184억원(+8.8%YoY), 영업이익 1조 8,994억원 (+27.6%YoY, OPM 18.1%)을 기록할 것으로 전망한다. 서치플랫폼 매출액은 3조 8,546억원 (+7.4%YoY)으로 추정한다. DA 광고 지면 증가 효과는 4분기까지 높은 성장률로 확인될 전 망이다. 커머스 매출액은 2조 9,045억원(+14.1%YoY)으로 전망한다. 네이버는 연내 모바일 쇼핑 탭을 네이버플러스 스토어로 개편할 계획이다. 스마트/브랜드스토어를 중심으로한 개 인화 추천, UI/UX 개선이 도입될 예정이다. 성장 둔화를 겪고 있는 커머스 광고 매출과 커 머스 GMV의 반등을 기대한다.'}, page_content='네이버는 2024년 연결 기준 영업수익 10조 5,184억원, 영업이익 1조 8,994억원을 기록할 것으로 전망하며, 특히 쇼핑 탭을 네이버플러스 스토어로 개편하여 개인화 추천 및 UI/UX 개선을 통해 커머스 광고 매출과 GMV의 반등을 기대함.'),
 Document(metadata={'id': 11, 'type': 'paragraph', 'image_route': nan, 'dir_route': 'datas/롯데렌탈_하나증권(2024.09.20)', 'file_name': '롯데렌탈_하나증권(2024.09.20).pdf', 'page': 2, 'investment

In [10]:
# 어째서인지 Ensemble retrieval은 topk가 먹질 않습니다.. ㅋㅋ
ensemble_retriever.invoke(query)

[Document(metadata={'company_name': '네이버', 'dir_route': 'datas/네이버_하나증권(2024.10.10)', 'file_name': '네이버_하나증권(2024.10.10).pdf', 'id': 19, 'investment': '하나증권(2024.10.10)', 'original_content': '네이버는 2024년 연결 기준 영업수익 10조 5,184억원(+8.8%YoY), 영업이익 1조 8,994억원 (+27.6%YoY, OPM 18.1%)을 기록할 것으로 전망한다. 서치플랫폼 매출액은 3조 8,546억원 (+7.4%YoY)으로 추정한다. DA 광고 지면 증가 효과는 4분기까지 높은 성장률로 확인될 전 망이다. 커머스 매출액은 2조 9,045억원(+14.1%YoY)으로 전망한다. 네이버는 연내 모바일 쇼핑 탭을 네이버플러스 스토어로 개편할 계획이다. 스마트/브랜드스토어를 중심으로한 개 인화 추천, UI/UX 개선이 도입될 예정이다. 성장 둔화를 겪고 있는 커머스 광고 매출과 커 머스 GMV의 반등을 기대한다.', 'page': 1, 'type': 'paragraph'}, page_content='네이버는 2024년 연결 기준 영업수익 10조 5,184억원, 영업이익 1조 8,994억원을 기록할 것으로 전망하며, 특히 쇼핑 탭을 네이버플러스 스토어로 개편하여 개인화 추천 및 UI/UX 개선을 통해 커머스 광고 매출과 GMV의 반등을 기대함.'),
 Document(metadata={'company_name': '네이버', 'dir_route': 'datas/네이버_하나증권(2024.10.10)', 'file_name': '네이버_하나증권(2024.10.10).pdf', 'id': 45, 'investment': '하나증권(2024.10.10)', 'original_content': '주: 2Q21 왓패드 인수, 4Q21

In [11]:
class WeightedRetriever:
    def __init__(self, dpr_weight=0.6, bm25_weight=0.4, top_k=2):
        # 가중치와 top_k 값을 초기화
        self.dpr_weight = dpr_weight
        self.bm25_weight = bm25_weight
        self.top_k = top_k

    def get_weighted_results(self, DPR_result, BM25_result):
        # 가중치를 적용한 결과를 저장할 딕셔너리 초기화 (ID를 키로 사용)
        weighted_results = {}

        # DPR 결과에 가중치 적용
        for doc in DPR_result:
            doc_id = doc.metadata['id']  # 문서의 id 추출
            score = self.dpr_weight  # DPR에 가중치 부여

            # 문서가 이미 존재하면 score를 합산, 아니면 새로 추가
            if doc_id in weighted_results:
                weighted_results[doc_id]['score'] += score
            else:
                weighted_results[doc_id] = {
                    'metadata': doc.metadata,
                    'score': score,  # 새로 추가된 score
                    'page_content': doc.page_content
                }

        # BM25 결과에 가중치 적용
        for doc in BM25_result:
            doc_id = doc.metadata['id']  # 문서의 id 추출
            score = self.bm25_weight  # BM25에 가중치 부여

            # 문서가 이미 존재하면 score를 합산, 아니면 새로 추가
            if doc_id in weighted_results:
                weighted_results[doc_id]['score'] += score
            else:
                weighted_results[doc_id] = {
                    'metadata': doc.metadata,
                    'score': score,  # 새로 추가된 score
                    'page_content': doc.page_content
                }

        # weighted_results에서 문서를 리스트로 변환하여 점수 기준으로 내림차순 정렬
        sorted_results = sorted(weighted_results.values(), key=lambda x: x['score'], reverse=True)

        # top_k 문서 선택
        top_k_documents = sorted_results[:self.top_k]

        return top_k_documents

    def display_results(self, DPR_result, BM25_result):
        top_k_documents = self.get_weighted_results(DPR_result, BM25_result)

        # 결과 출력
        for doc in top_k_documents:
            print(f"ID: {doc['metadata']['id']}, Score: {doc['score']:.4f}")
            print(f"Content: {doc['page_content']}\n")


In [13]:
dpr_weight = 0.6
bm25_weight = 0.4
top_k = 3

# 인스턴스 생성
weighted_retriever = WeightedRetriever(dpr_weight=dpr_weight, bm25_weight=bm25_weight, top_k=top_k)

# query에 대해 DPR과 BM25 결과를 가져옴 (예시)
DPR_result = DPRRetriever.invoke(query)
BM25_result = BM25Retriever.invoke(query)

# 결과 출력
weighted_retriever.display_results(DPR_result, BM25_result)


ID: 19, Score: 1.0000
Content: 네이버는 2024년 연결 기준 영업수익 10조 5,184억원, 영업이익 1조 8,994억원을 기록할 것으로 전망하며, 특히 쇼핑 탭을 네이버플러스 스토어로 개편하여 개인화 추천 및 UI/UX 개선을 통해 커머스 광고 매출과 GMV의 반등을 기대함.

ID: 45, Score: 0.6000
Content: 네이버의 2021년 2분기 매출액은 전년동기 대비 30.4% 증가한 1조 6,635억원이며 영업이익은 8,362억원으로 전년동기 대비 16.2% 감소했다. 이는 21년 2분기에 있었던 왓패드와 이북 재팬 인수 및 22년 4분기부터 적용된 순매출 > 총매출 회계 처리 방식 변경 때문이다.

ID: 13, Score: 0.6000
Content: 네이버는 개인화 앱 개편 영향으로 서치 플랫폼 성장률이 가속화되고 있으며, 자체 경쟁력 강화로 인한 성장 추세는 계속될 것으로 예상된다. 또한 'AI 브리핑' 도입으로 사용성이 개선되며, 다양한 전략을 통해 2025년 커머스 GMV 10.3% 성장을 전망한다.



In [14]:
from evaluation import retrieval_evaluate
import pandas as pd
eval_dataset = pd.read_csv('../../qa_validation_dataset_3_cleaning_2.csv')

# retrieval과 evaldataset을 통해 acc를 평가해볼 수 있습니다.
retrieval_evaluate(retrieval=DPRRetriever, eval_dataset = eval_dataset)

acc: 0.8577


In [27]:
def weighted_retrieval_evaluate(weighted_retriever, eval_dataset):
    total = len(eval_dataset)
    answer = 0
    for i, row in eval_dataset.iterrows():
        question = row['question']
        context = row['context']
        
        # weighted_retriever로 검색 결과 얻기
        retrieval_results = weighted_retriever.get_weighted_results(DPR_result=DPRRetriever.invoke(question), BM25_result=BM25Retriever.invoke(question))
        
        # 검색된 문서에서 내용만 추출하여 평가
        docs = [doc['metadata']['original_content'] for doc in retrieval_results]
        
        # context가 검색된 문서에 포함되어 있으면 정답
        if context in docs:
            answer += 1
    
    print('acc:', round(answer / total, 4))


In [28]:
weighted_retrieval_evaluate(weighted_retriever=weighted_retriever, eval_dataset=eval_dataset)

acc: 0.8577
